# Lab 5 - Dependency parsing
Laboration 5 in EDAN20 @ LTH - http://cs.lth.se/edan20/coursework/assignment-5/

Author: Jonatan Kronander

## Objectives
The objectives of this assignment are to:
* Know what a dependency graph is
* Understand the principles of Nivre's parsing mechanisms
* Extend Nivre's parser with a guiding predicate that parses an annotated dependency graph
* Extract features to learn parsing actions from an annotated corpus
* Write a short report on your results
* In this assignment, you will only generate the machine-learning models from the extracted features. You will complete the parser and apply it in the next assignment.

This assignment is inspired by the shared task of the Tenth conference on computational natural language learning,CONLL-X, and uses a subset of their data. The conference site contains a description of multilingual dependency parsing, reference papers, training and test sets for a variety of languages, as well as evaluation programs. See also CONLL 2007, on the same topic.

Please note that the original CoNLL-X site is down. To access the pages, use the Archive.org site: https://web.archive.org/web/20161105025307/http://ilk.uvt.nl/conll/ and to download the data sets, use the local copies.

In this session, you will implement a dependency parser for Swedish. Should you want to use another corpus, please tell me in advance.

## Choosing a training and a test sets

Use the CONLL-X Swedish corpus. Download the tar archives containing the training and test sets for Swedish and uncompress them: 
http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/conllx/sv/swedish_talbanken05_train.conll http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/conllx/sv/swedish_talbanken05_test_blind.conll http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/conllx/sv/swedish_talbanken05_test.conll

In [2]:
from urllib.request import urlopen

b_train_text = urlopen("http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/conllx/sv/swedish_talbanken05_train.conll").read() # Open file and read
train_text = str(b_train_text,'utf-8')

b_test_text = urlopen("http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/conllx/sv/swedish_talbanken05_test.conll").read() # Open file and read
test_text = str(b_test_text,'utf-8')

b_blind_test_text = urlopen("http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/conllx/sv/swedish_talbanken05_test_blind.conll").read() # Open file and read
blind_test_text = str(b_blind_test_text,'utf-8')

In [9]:
f_out = open('train.conll', 'w')

for sentence in train_text:   
    f_out.write(sentence)
f_out.close()

f_out = open('test.conll', 'w')

for sentence in test_text:   
    f_out.write(sentence)
f_out.close()

f_out = open('test_blind.conll', 'w')

for sentence in blind_test_text:   
    f_out.write(sentence)
f_out.close()

## Nivre's parser

For each sentence with a projective dependency graph, there is an action sequence that enables Nivre's parser to generate this graph. Gold standard parsing corresponds to the sequence of parsing actions, left-arc (la), right-arc (ra), shift (sh), and reduce (re) that produces the manually-obtained, gold standard, graph.

Using an annotated corpus, we can derive all the action sequences producing the manually-parsed sentences (provided that they are projective). We can then train a classifier to predict an action from a current parsing context. To be able to predict the next action from a given parsing state, gold standard parsing must also extract feature vectors at each step of the parsing procedure. The simplest parsing context corresponds to words' part of speech on the top of the stack and head of the input list (the queue).

Once the data collected, the training procedure will produce a 4-class classifier that you will embed in Nivre's parser to choose the next action. During parsing, Nivre's parser will call the classifier to choose the next action in the set {la, ra, sh, re} using the current context.

#### 1.Run the dparser.py program.

In [10]:
import subprocess

cmd_1 = "python dparser.py"
print(cmd_1)

p = subprocess.Popen(cmd_1, stdout=subprocess.PIPE, shell=True)
out_1, err = p.communicate() 
print(str(out_1,'utf-8'))

python dparser.py


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
out = str(out_1,'utf-8')

In [13]:
print(out[:500])

Equal graphs: True
['sh', 'sh', 'sh', 'la.++', 'ra.CC', 're', 'la.SS', 'ra.ROOT', 'sh', 'sh', 'la.AT', 'la.DT', 'ra.SP', 'ra.IK', 'sh', 'la.SS', 're', 'ra.ET', 'ra.TA', 'ra.PA', 're', 're', 're', 're', 'ra.IP']
{'heads': {'0': '0', '2': '3', '3': '1', '1': '4', '4': '0', '6': '7', '5': '7', '7': '4', '8': '7', '9': '10', '10': '7', '11': '10', '12': '11', '13': '4'}, 'deprels': {'0': 'ROOT', '2': '++', '3': 'CC', '1': 'SS', '4': 'ROOT', '6': 'AT', '5': 'DT', '7': 'SP', '8': 'IK', '9': 'SS', '10'


#### 2. Understand from the slides and the program how Nivre's parser is extended to carry out a gold standard parsing. Given a manually-annotated dependency graph, what are the conditions on the stack and the current input list -- the queue -- to execute left-arc, right-arc, shift, or reduce? Start with left-arc and right-arc, which are the simplest ones. https://github.com/pnugues/ilppp/blob/master/slides/EDAN20_ch13.pdf